In [1]:
import os
import sys
sys.path.append('../')
os.chdir('../')

In [2]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import duck_db_helper
import duckdb

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
conn = duckdb.connect('../data/feature_generation/train.db')
tables = duck_db_helper.get_tables(conn)
df_list = []
df_list_names = []
for table_name in tables: 
    df = conn.sql(f"SELECT * FROM \"{table_name}\";").df()
    df_list.append(df)
    df_list_names.append(table_name)
conn.close()
df = df_list[0]

In [7]:
model = smf.ols("total_points ~ name*(TEMP+odds_for)", data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           total_points   R-squared:                       0.276
Model:                            OLS   Adj. R-squared:                  0.133
Method:                 Least Squares   F-statistic:                     1.931
Date:                Tue, 12 Dec 2023   Prob (F-statistic):           8.04e-46
Time:                        09:46:12   Log-Likelihood:                -13636.
No. Observations:                5844   AIC:                         2.920e+04
Df Residuals:                    4881   BIC:                         3.562e+04
Df Model:                         962                                         
Covariance Type:            nonrobust                                         
                                                           coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------

In [9]:
model.save("../data/models/model1.pickle")
# create a database if it doesn't exist, that is the model registry. 
# name_of_model | dataset_used | date_trained